In [90]:
import pandas as pd
import scipy
from scipy.optimize import minimize
import numpy as np


props_df = pd.DataFrame(
    { 
        "A":[1., 1., 1.], 
        "B":[0., 0., 0.], 
        "C":[0., .0, .0]
    },
    index = [0, 100 , 150]
)

active_flows = [
    "A_to_B", 
    "B_to_C", 
    "C_to_B"
]


def calculate_transition_rates_from_dynamic_props(props_df, active_flows):      
    strata = props_df.columns.to_list()
    times = props_df.index.to_list()
    
    tv_rates = {flow : [] for flow in active_flows}
    print(active_flows)
    for i in range(len(times) - 1):
        delta_t = times[i + 1] - times[i]
        start_props = props_df.loc[times[i]]
        end_props = props_df.loc[times[i + 1]]
        
        rates = calculate_rates_for_interval(start_props, end_props, delta_t, strata, active_flows)      
        for i_flow, flow in enumerate(active_flows):
            tv_rates[flow].append(rates[i_flow])
            
    print(tv_rates)
    
def calculate_rates_for_interval(start_props, end_props, delta_t, strata, active_flows):
    n_strata = len(strata)
    n_params = len(active_flows)
    
    def function_to_zero(params):        
        # params are ordered in the same order as active_flows
       
        m = np.zeros((n_strata, n_strata))        
        
        for i_row, stratum_row in enumerate(strata):
            for i_col, stratum_col in enumerate(strata):
                if i_row == i_col:
                    relevant_flows = [f for f in active_flows if f.startswith(f"{stratum_row}_to_")]
                    for f in relevant_flows:
                        m[i_row, i_col] -= params[active_flows.index(f)]
                else:
                    potential_flow = f"{stratum_col}_to_{stratum_row}"
                    if potential_flow in active_flows:
                        m[i_row, i_col] = params[active_flows.index(potential_flow)]
                        
        exp_mt = scipy.linalg.expm(m * delta_t)
        diff = np.matmul(exp_mt, start_props) - end_props
        
        return scipy.linalg.norm(diff)    
    
    bounds = [(0., None)] * n_params
    solution = minimize(function_to_zero, x0=np.zeros(n_params), bounds=bounds)
    
    return solution.x
        
calculate_transition_rates_from_dynamic_props(props, active_flows)

['A_to_B', 'B_to_C', 'C_to_B']
{'A_to_B': [0.004700030859341583, 0.22920280892437866], 'B_to_C': [0.003156966610331139, 0.04688094936343767], 'C_to_B': [1.0175118407444169e-05, 0.04584384387186252]}


In [85]:
0.8 * np.exp(-4.70003086e-03 * 100)

0.5000002716229416